# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

Import the dataset with:
- mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json


In [107]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [108]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [109]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [110]:
# assign the uk_food database to a variable name
uk_food_db = mongo['uk_food']

In [111]:
# review the collections in our new database
print(uk_food_db.list_collection_names())

['establishments']


In [112]:
# review a document in the establishments collection
uk_food_db.establishments.find_one()

{'_id': ObjectId('666726b2e9ec2694357a6831'),
 'FHRSID': 1290215,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DH',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2020-11-09T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.378231', 'latitude': '51.150625'},
 'RightToReply': '',
 'Distance': 4587.64041272672,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  

In [113]:
# assign the collection to a variable
establishments = uk_food_db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [114]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [115]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('666726fa43f14a12542e08ab'), acknowledged=True)

In [116]:
# Check that the new restaurant was inserted
new_restaurant_find = list(establishments.find({"BusinessName":"Penang Flavours"}))
# Check to see that we only have the one new entry
print(len(new_restaurant_find))
# Display
pprint(new_restaurant_find)

1
[{'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': '',
  'Distance': 4623.972328074718,
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': True,
  'Phone': '',
  'PostCode': 'SE18 7DY',
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('666726fa43f14a12542e08ab'),
  'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
  'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}]


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [117]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
BusinessTypeID_find = list(establishments.find({"BusinessType":"Restaurant/Cafe/Canteen"},{"BusinessTypeID":1,"BusinessType":1}))
pprint(BusinessTypeID_find[0])

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('666726b2e9ec2694357a6831')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [118]:
# Update the new restaurant with the correct BusinessTypeID
verify_update = establishments.update_one(new_restaurant,{"$set":{"BusinessTypeID":1}})
print(verify_update)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)


In [119]:
# Confirm that the new restaurant was updated
update_verify = list(establishments.find({"BusinessName":"Penang Flavours"},{"BusinessTypeID":1, "BusinessName":1}))

print(f'Count of records changed: {len(update_verify)}')
for uv in update_verify:
    pprint(uv)

Count of records changed: 1
{'BusinessName': 'Penang Flavours',
 'BusinessTypeID': 1,
 '_id': ObjectId('666726fa43f14a12542e08ab')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [120]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName":{"$regex":"Dover"}})
print(dover_count)

994


In [121]:
# Delete all documents where LocalAuthorityName is "Dover"
dover_delete = establishments.delete_many({"LocalAuthorityName":{"$regex":"Dover"}})
print(dover_delete)

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)


In [122]:
# Check if any remaining documents include Dover
dover_count = establishments.count_documents({"LocalAuthorityName":{"$regex":"Dover"}})
print(dover_count)

0


In [123]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('666726b2e9ec2694357a6b1a'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [124]:
# Change the data type from String to Decimal for longitude and latitude
lon_lat_update = establishments.update_many({}, [{'$set': {'geocode.longitude': {'$toDouble': '$geocode.longitude'}, 'geocode.latitude': {'$toDouble': '$geocode.latitude'}}}])
print(lon_lat_update)

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)


Use `update_many` to convert `RatingValue` to integer numbers.

In [125]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
update_rslts = establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])
print(update_rslts)

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)


In [126]:
# Change the data type from String to Integer for RatingValue
ratingvalue_update = establishments.update_many({}, [{'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}])
print(ratingvalue_update)

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)


In [127]:
# Check that the coordinates and rating value are now numbers
rating_changes_int_count = establishments.count_documents({"RatingValue":{"$gte":0}})
rating_changes_str_count = establishments.count_documents({"RatingValue":{"$gte":'0'}})
rating_changes_none_count = establishments.count_documents({"RatingValue":{"$eq":None}})

print(f'{rating_changes_int_count} ratings stored as integers \n \
{rating_changes_str_count} ratings stored as strings \n \
{rating_changes_none_count} empty ratings \n')

coordinates_num_count = establishments.count_documents({"geocode.longitude":{"$gte":0}})
coordinate_str_count = establishments.count_documents({"geocode.latitude":{"$gte":'0'}})

print(f'{coordinates_num_count} geocodes stored as integers \n {coordinate_str_count} geocodes stored as strings')

34694 ratings stored as integers 
 0 ratings stored as strings 
 4092 empty ratings 

38786 geocodes stored as integers 
 0 geocodes stored as strings
